# Description
Converts the [CMMC] control catalog to [OpenControl] format.
## Data source
- https://www.acq.osd.mil/cmmc/docs/CMMCModelExcel_V1.02_20200318.xlsx

[CMMC]: https://www.acq.osd.mil/cmmc
[OpenControl]: https://open-control.org/

In [1]:
import pandas as pd
import yaml
from itertools import groupby

In [2]:
dfs = pd.concat(pd.read_excel(
    'CMMCModelExcel_V1.02_20200318.xlsx',
    sheet_name=list(range(2,19)),
    header=[0,2],
    index_col=[0],
))
dfs.columns.names = ['domain','level']
dfs.index.names = ['row','capability']

In [3]:
df = (dfs
      .melt(ignore_index=False,value_name='practice')
      .dropna()
      .reset_index()
      .drop('row', axis=1)
     )
df.head()

,capability,domain,level,practice
0,C001\nEstablish system access requirements,DOMAIN: ACCESS CONTROL (AC),Level 1 (L1),AC.1.001\nLimit information system access to a...
1,C002\nControl internal system access,DOMAIN: ACCESS CONTROL (AC),Level 1 (L1),AC.1.002\nLimit information system access to t...
2,C004\nLimit data access to authorized users an...,DOMAIN: ACCESS CONTROL (AC),Level 1 (L1),AC.1.003\nVerify and control/limit connections...
3,C004\nLimit data access to authorized users an...,DOMAIN: ACCESS CONTROL (AC),Level 1 (L1),AC.1.004\nControl information posted or proces...
4,C001\nEstablish system access requirements,DOMAIN: ACCESS CONTROL (AC),Level 2 (L2),AC.2.005\nProvide privacy and security notices...


## Verification
Verify number of practices at each level.

**Expected results:** (Ref: CMMC section 2.7.1 Figure 5 _CMMC Practices Per Level_)

| Level | Count | Total |
| --- | --- | --- |
| ML1 |  17 |  17 |
| ML2 |  55 |  72 |
| ML3 |  58 | 138 |
| ML4 |  26 | 156 |
| ML5 |  15 | 171 |


In [4]:
df.groupby('level').count().practice

level
Level 1 (L1)     17
Level 2  (L2)    55
Level 3 (L3)     58
Level 4 (L4)     26
Level 5 (L5)     15
Name: practice, dtype: int64

## Tidy up and reformat


In [5]:
def reformatDomain(domain):
    """Change 'DOMAIN: XXX AND YYY (XY)' to 'Xxx and Yyy (XY)'"""
    return (
        domain[:-5]
        .replace('DOMAIN: ', '')
        .title()
        .replace('And', 'and')
    ) + domain[-5:]

def parsePractice(cell):
    """Split spreadsheet cell into ID, description, and mappings"""
    lines = cell.split('\n')
    return {
        'id': lines[0],
        'description': ' '.join([l for l in lines[1:] if not(isMappingLine(l))]),
        'mapping': [l[2:] for l in lines[1:] if isMappingLine(l)],
    }

def isMappingLine(line):
    """Distinguish control mapping lines from other text"""
    return line.startswith('\u2022')

In [6]:
db = (
    df
    .assign(domain = df.domain.apply(reformatDomain))
    .assign(level = df.level.str.split(expand=True)[1])
    .join(df.practice.apply(parsePractice).apply(pd.Series))
    .drop('practice',axis=1)
    .set_index(['domain','capability','id'])
    .sort_index(level=['capability','id'])
)
db.head()

level  \
domain              capability                                 id               
Access Control (AC) C001\nEstablish system access requirements AC.1.001     1   
                                                               AC.2.005     2   
                                                               AC.2.006     2   
                    C002\nControl internal system access       AC.1.002     1   
                                                               AC.2.007     2   

                                                                                                               description  \
domain              capability                                 id                                                            
Access Control (AC) C001\nEstablish system access requirements AC.1.001  Limit information system access to authorized ...   
                                                               AC.2.005  Provide privacy and security notices consisten...   
                                                               AC.2.006  Limit use of portable storage devices on exter...   
                    C002\nControl internal system access       AC.1.002  Limit information system access to the types o...   
                                                               AC.2.007  Employ the principle of least privilege, inclu...   

                                                                                                                   mapping  
domain              capability                                 id                                                           
Access Control (AC) C001\nEstablish system access requirements AC.1.001  [FAR Clause 52.204-21 b.1.i, NIST SP 800-171 R...  
                                                               AC.2.005  [NIST SP 800-171 Rev 1 3.1.9, NIST SP 800-53 R...  
                                                               AC.2.006  [NIST SP 800-171 Rev 1 3.1.21, CIS Controls v7...  
                    C002\nControl internal system access       AC.1.002  [FAR Clause 52.204-21 b.1.ii, NIST SP 800-171 ...  
                                                               AC.2.007  [NIST SP 800-171 Rev 1 3.1.5, CIS Controls v7....

# Convert to OpenControl format

## Extract control catalog

In [7]:
def reformatMapping(mapping):
    """Reformat list of mapping records"""
    return 'References: ' + '; '.join(mapping)

In [8]:
name = 'CMMC-1.0'
catalog = (
    db
    .assign(mapping = db.mapping.apply(reformatMapping))
    .reset_index(level=['domain','capability'])
    .rename(columns={
        'domain': 'family',
        'description': 'name',
        'mapping': 'description',
    })
    .loc[:,['name', 'family', 'description']]
    .to_dict('index')
)
catalog['name'] = name

In [9]:
with open(name + '.yaml', 'w') as outfile:
    yaml.dump(catalog, outfile)

## Extract profiles

In [10]:
db4 = db.reset_index(['domain','capability'],drop=True)
for level in ['1','3']:
    controls = db4.loc[db4.level <= level,[]]
    pname = "%s-ML%s" % (name,level)
    profile = {
        "name" : pname,
        "standards" : {
            name : controls.to_dict('index')
        }
    }
    with open(pname + '.yaml','w') as outfile:
        yaml.dump(profile, outfile)

# Extract mappings

## Notes
- There are some inconsistencies in the original spreadsheet (e.g., `NIST 800-53 Rev 4` vs `NIST SP 800-53 Rev 4`).
- CMMC 1.02 (2020-03-18) references expired draft NIST SP 800-171B; superseded by 800-172

The `standards` dictionary maps them to names consistent with other OpenControl artifacts.

In [11]:
standards = {
    'CERT RMM v1.2': 'CERT RMM',
    'CIS Controls v7.1': 'CIS Controls',
    'CMMC modification of Draft NIST SP 800-171B': 'NIST-800-172',
    'CMMC ': 'CMMC', # account for "CMMC modification of ..."
    'FAR Clause': 'FAR', # FAR Clause 52.204-21
    'NIST 800-53 Rev 4': 'NIST-800-53',
    'NIST SP 800-53 Rev 4': 'NIST-800-53',
    'NIST SP 800-171 Rev 1': 'NIST-171r1',
    'Draft NIST SP 800-171B': 'NIST-800-172',
    'NIST CSF v1.1': 'NIST CSF',
    'NIST CSF v1.2': 'NIST CSF',
}
def parseMapping(m):
    """Split mapping into source standard and list of controls"""
    global standards
    for s, r in standards.items():
        if m.startswith(s):
            return { 'standard': r, 'controls': m.replace(s,'') }
    return { 'standard': 'Other', 'controls': m }

def splitControls(controls):
    """Split comma-separated list of controls"""
    return [c.strip().replace('(',' (') for c in controls.split(',')]

In [12]:
selection = pd.DataFrame({ 'standard': [
    'NIST-800-53',
    'NIST-800-171r1',
]})

In [13]:
map_table = (
    db.loc[:,['mapping']]
    .explode('mapping')
    .apply(lambda m: pd.Series(parseMapping(m.mapping)), axis=1)
)
# map_table.head()

## Verification

In [14]:
# Examine which mappings are identified as "Other":
map_table.loc[map_table.standard == 'Other'].controls.unique()

array(['AU ACSC Essential Eight', 'UK NCSC Cyber Essentials', 'CMMC',
       'UK NCSC Cyber Essentials '], dtype=object)

In [15]:
# Examine CMMC-specific mappings:
map_table.loc[map_table.standard == 'CMMC'].controls.unique()

array(['modification of NIST SP 800-171 3.4.8', '',
       'modification of NIST SP 800-171 Rev 1 3.13.2'], dtype=object)

## Convert
Control mapping is saved using an `ssptool` extension to OpenControl `certification` schema.

In [16]:
recs = (
    map_table
    .assign(controls = map_table.controls.apply(splitControls))
    .reset_index(['domain','capability'],drop=True)
    .reset_index(drop=False)
    .merge(selection, on='standard')
    .explode('controls')
    .to_dict('records')
)

In [17]:
recs_nested = {
    n: { 'mapping': {
        s : [e['controls'] for e in v]
        for s, v in groupby(g, lambda x: x['standard']) 
    }}
    for n, g in groupby(recs, lambda x: x['id'])
}

In [18]:
pname = name + '-mapped'
profile = {
    'name': pname,
    'standards': {
        name : recs_nested
    }
}

In [19]:
with open(pname + '.yaml', 'w') as outfile:
    yaml.dump(profile,outfile)